In [53]:
import pandas as pd
import re
from collections import Counter

pattern1 = re.compile(r'[+/]')
pattern2 = re.compile(r'\(\w+\)')

def split_and_clean_course_code(l):
    if type(l) != str:
        return []
    tmp = pattern1.sub(r',', l)
    tmp = pattern2.sub(r'', tmp)
    tmp = tmp.strip().strip(',')
    tmp = [c.strip() for c in tmp.split(',')]
    return tmp

In [54]:
Projections = {
                'Old': ['StudentsAudit_Fatna.xlsm','STUD_ID','Manual Audit'],
                'New': ['StudentsAudit.xlsx','ID','Audit_Courses_to_add'],
                # 'New': ['StudentsAudit_old.xlsx','ID','Audit_Courses_to_add']
}

In [55]:
old = pd.read_excel(Projections['Old'][0])[Projections['Old'][1:]]
new = pd.read_excel(Projections['New'][0])[Projections['New'][1:]]
old.columns = ['ID','old']
new.columns = ['ID','new']
old['oID'] = old.ID
new['nID'] = new.ID
df = pd.merge(old, new, on='ID', how='outer')

count = len(df)
mask1 = df.oID.isnull() & df.nID.notnull()
mask2 = df.oID.notnull() & df.nID.isnull()
count1 = len(df[mask1])
count2 = len(df[mask2])
if count1:
    print(f'Found {count1} in new but missing from old ({count1/count*100:.2f}%). Excluding the records.')
    # print(df[mask1])
    df.drop(df[mask1].index, inplace=True)
if count2:
    print(f'\nFound {count2} in old but missing from new ({count2/count*100:.2f}%). Excluding the records.')
    # print(df[mask2])
    df.drop(df[mask2].index, inplace=True)
df.drop(['oID','nID'],inplace=True,axis=1)

print(f'\n{len(df)} records left after cleaning.\n')

Found 642 in new but missing from old (39.65%). Excluding the records.

977 records left after cleaning.



In [56]:
df.old = df.old.apply(split_and_clean_course_code)
df.new = df.new.apply(split_and_clean_course_code)
df['Delta_old'] = pd.Series(dtype='object')
df['Delta_new'] = pd.Series(dtype='object')
df['Match'] = pd.Series(dtype='float')

for i, r in df.iterrows():
    o = Counter(r.old)
    n = Counter(r.new)
    Delta_o_n = list((o-n).elements())
    Delta_n_o = list((n-o).elements())
    df.at[i,'Delta_old'] = Delta_o_n
    df.at[i,'Delta_new'] = Delta_n_o
    if len(r.old)+len(r.new) == 0:
        df.at[i,'Match'] = 1
    else:
        df.at[i,'Match'] = 1 - (len(Delta_o_n)+len(Delta_n_o)) / (len(r.old)+len(r.new))

print(f'\ncompared {len(df)} students, with similarity ratio: {df.Match.sum()/len(df)*100:.2f}%')
print(f'\n{len(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])} differences:')
# display(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ])


compared 977 students, with similarity ratio: 94.18%

177 differences:


In [57]:
# display(df[ (df.Delta_old.map(len)>0) | (df.Delta_new.map(len)>0) ][0:50])
# display( df[df['old'].astype(str).str.contains(')',regex=False)] )


In [58]:
cat = ['Status', 'Standing', 'Campus', 'Audit_UsedCatalog', 'College', 'Degree', 'Major', 'Concentration', 'Program']

data = pd.read_excel(Projections['New'][0])
df2 = pd.merge(data, df, on='ID', how='inner')
for c in cat:
    display(df2.groupby(c)['Match'].agg(['count','mean']))

,count,mean
Status,,
Academic Dismissal (DS),4,0.778409
Active (AS),865,0.941967
Attendance Suspension (AT),3,0.933333
Maternity Suspension (MA),2,0.944444
Military Service (MS),12,0.966667
Military Svc Completion (MC),58,1.000000
Military Svc Exempted (ME),13,1.000000
Registration Suspension (RS),9,0.833333
Reinstated (RE),1,0.000000


,count,mean
Standing,,
Academic Dismissal (AD),4,0.778409
Academic Probation 1 (P1),5,0.634343
Academic Probation 1 (PO),4,0.375000
Academic Probation 2 (P2),10,0.661364
Academic Probation 2 (PT),4,0.791667
Academic Warning (AW),62,0.938238
Academic Warning 1 (W1),16,0.792140
Academic Warning 2 (W2),1,0.400000
Good Standing (00),6,0.833333


,count,mean
Campus,,
AUH - Men's Campus,164,0.960994
AUH - Women's Campus,181,0.977158
AUH Mixed Gender,632,0.926628


,count,mean
Audit_UsedCatalog,,
2019,58,0.967476
2020,139,0.975267
2021,337,0.961732
2024,303,0.993399
2024_CIS,139,0.743297


,count,mean
College,,
Technological Innovation (TI),977,0.941758


,count,mean
Degree,,
Bachelor of Science (BS),977,0.941758


,count,mean
Major,,
Info Systems & Tech Management (BIST),278,0.934184
Information Technology (BINT),699,0.944771


,count,mean
Concentration,,
Business Intelligence (_BI),201,0.932347
Enterprise Systems (_ES),6,0.966667
Mgmt. of Information Systems (_MS),71,0.936639
Security and Network Tech (_SN),637,0.944072
Web and Mobile App Development (_WM),62,0.951948


,count,mean
Program,,
TI_BSINT,699,0.944771
TI_BSIST,278,0.934184


In [59]:
df2.to_excel('Compare_Projections.xlsx')
